<div>
    <h1 align="center">Tabular Playground Series - May 2021</h1>
</div>

<div class="alert alert-success">  
</div>

## Import

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<div class="alert alert-success">  
</div>

## Data Set

In [ ]:
df1 = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
df2 = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
sam = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")

In [ ]:
# display(df1.info())
# df1.describe().transpose()

In [ ]:
MV = df1.isnull().sum()
print(f'Missing Value:\n{MV[MV > 0]}')

In [ ]:
df1['target'].value_counts(normalize=True).plot(kind='bar')
df1['target'].value_counts()

In [ ]:
data1 = df1.copy()
dic = {"Class_1":0, "Class_2":1, "Class_3":2, "Class_4":3}
data1['target'].replace(dic, inplace=True)

In [ ]:
X = data1.drop(columns = ['id','target'])
display(X)

In [ ]:
y = data1.target
display(y)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.20, random_state=123) 

In [ ]:
data2 = df2.copy()
XX = data2.drop(columns = ['id'])
display(XX)

<div class="alert alert-success">  
</div>

## CatBoostClassifier()

In [ ]:
model1 = CatBoostClassifier(iterations=17000,
                            learning_rate=0.01,
                            depth=4,
                            loss_function='MultiClass',
                            od_wait=1000,
                            od_type='Iter',                           
                            min_data_in_leaf=1,
                            max_ctr_complexity=15,
                            task_type='GPU',
                            devices='0:1')                            
                            
model1.fit(X, y, verbose=1000)
pred1 = model1.predict_proba(XX)
pred1 = np.clip(pred1, 0.08, 0.95)
display(pred1) 

In [ ]:
model1.feature_importances_

In [ ]:
sub1 = sam.copy()
display(sub1, pred1, pred1.shape)

In [ ]:
sub1['Class_1'] = pred1[:,0] 
sub1['Class_2'] = pred1[:,1] 
sub1['Class_3'] = pred1[:,2] 
sub1['Class_4'] = pred1[:,3] 
display(sub1)

In [ ]:
sub1.to_csv("submission1.csv",index=False)
# Public Score: 1.08627

<div class="alert alert-success">  
</div>

## TabularUtilizedAutoML()

Thanks to: @alexryzhkov
https://www.kaggle.com/alexryzhkov/lightautoml-baseline-tps-may-2021

In [ ]:
sub2 = pd.read_csv('../input/lightautoml-baseline-tps-may-2021/lightautoml_2variants_ensemble.csv')
display(sub2)

In [ ]:
sub2.to_csv("submission2.csv",index=False)
# Public Score: 1.08593

<div class="alert alert-success">  
</div>

In [ ]:
def generate(main, support, coeff):
    
    g = main.copy()    
    for i in main.columns[1:]:
        
        res = []
        lm, Is = [], []        
        lm = main[i].tolist()
        ls = support[i].tolist()  
        
        for j in range(len(main)):
            res.append((lm[j] * coeff) + (ls[j] * (1.- coeff)))            
        g[i] = res
        
    return g

In [ ]:
sub = generate(sub2, sub1, 0.60)
display(sub)

In [ ]:
sub.to_csv("submission.csv",index=False)
!ls

<div class="alert alert-success">
    <h1 align="center">If you find this work useful, please don't forget upvoting :)</h1>
    
    
    
    
</div>